# The Skew-T Log-p Diagram


In [ ]:
from datetime import datetime

import matplotlib.pylab as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import metpy.calc as mpcalc
from metpy.plots import Hodograph, SkewT
from metpy.units import units, pandas_dataframe_to_unit_arrays
import numpy as np
from siphon.simplewebservice.wyoming import WyomingUpperAir 

In [ ]:
date = datetime(2011, 4, 27, 0)
station = 'BMX'
df = WyomingUpperAir.request_data(date, station)
data_with_units = pandas_dataframe_to_unit_arrays(df)

# Only want to pull in data up to 100 mb, we can 
# use a boolean array to choose on the desired values
ip100 = data_with_units['pressure'] >= 100 * units.hPa

# Subset pressure and read in other data to only 100 mb
p = data_with_units['pressure'][ip100]
T = data_with_units['temperature'][ip100]
Td = data_with_units['dewpoint'][ip100]
u = data_with_units['u_wind'][ip100]
v = data_with_units['v_wind'][ip100]

## Plotting the Skew-T

MetPy has the ability to make a few specialty plots (with matplotlib as a backend). The skew-T plots are relatively easy to do with only a few lines to plot the key variables and lines on the plot. The main Class that we have to draw a sounding is the `SkewT` object. Here we have rotated the axes and have the ability to draw lines with the `skew.plot()` method, draw wind barbs with the `skew.plot_barbs` method and even plot the special lines of dry adiabats, moist adiabats, and mixing ratio.

https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.SkewT.html#metpy.plots.SkewT

In [ ]:
# Change default to be better for skew-T
fig = plt.figure(figsize=(10, 10))


# Plot the data using normal plotting functions, in this case using
# log scaling in Y, as dictated by the typical meteorological plot


# Add the relevant special lines


# Set title using the date and station variables directly


# Show the plot
plt.show()

### Refined Skew-T Plots

Draw better dry adiabats, moist adiabats, and mixing ratio lines (my defaults).

https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.SkewT.html#metpy.plots.SkewT

Add text and markers to the plot using the `text` and `plot` methods available on the skew-T object main axes `skew.ax.text` and `skew.ax.plot`. Accessing these base-level Matplotlib methods will work just the same as if you were plotting a typical x-y plot, but instead using pressure and temperature to plot appropriately on our skewed axis. Note that the ordering is reversed from our `SkewT` class object in that the x-value goes first (temperature) and the y-value (pressure) goes second.

## Calculate Parcel Profile

Now we want to use the MetPy function `parcel_profile` to calculated the surfaced-baed CAPE (SBCAPE). We'll need to feed the function with all of the pressure values along with a single starting temperature and dewpoint value. The starting temperature and dewpoint values should be the first value available in each array.

https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.parcel_profile.html#metpy.calc.parcel_profile

In [ ]:
parcel_temp = mpcalc.parcel_profile(p, T[0], Td[0])

In [ ]:
print(parcel_temp)

## Add Parcel Profile to Skew-T and Fill CAPE Area

We can start from our improved skew-T that we generated above, then add a new `skew.plot()` to plot the parcel temperature profile for each level in the pressure variable. Additionally we can use the method available in a `SkewT()` object to shade CAPE values `skew.shade_cape()`. A similar process works for CIN using the appropriate shade method.

https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.SkewT.html#metpy.plots.SkewT.shade_cape

## Hodograph Addition
We can also use the `Hodograph()` plot functionality to add that to our skew-T diagram. Here we'll demonstrate adding it as an inset axes within out skew-T plot itself. Note that Hodographs are plotted by height (not pressure) and we'll add a few nice elements to make our hodograph more interpretable.

https://unidata.github.io/MetPy/latest/api/generated/metpy.plots.Hodograph.html